In [15]:
from pyAudioAnalysis import audioFeatureExtraction as aF
from pyAudioAnalysis import audioTrainTest as aT
from pyAudioAnalysis import audioBasicIO

In [26]:
modelName="/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/Model_training_clips/svm_King_UBM_900x2s"

[Classifier, MEAN, STD, classNames, mtWin, mtStep, stWin, stStep, computeBEAT] = aT.loadSVModel(modelName)

stStep

0.05

In [27]:
fileName="/Volumes/Turcich-2012/AAPB_Test_Haystack_16000/cpb-aacip-500-4m91d343.16000.wav"

outPutFile="/Users/mclaugh/Desktop/af_test.csv"

[Fs, x] = audioBasicIO.readAudioFile(fileName)

mtWin=1.0
stWin=0.05

midTermSize = mtWin * Fs
midTermStep = 1.0

shortTermSize = stWin * Fs
shortTermStep = 0.05

#aF.mtFeatureExtraction(x, Fs, mtWin * Fs, mtStep * Fs, round(Fs * stWin), round(Fs * stStep))
#mtFeatureExtraction(signal, Fs, mtWin, mtStep, stWin, stStep):

aF.mtFeatureExtractionToFile(fileName, midTermSize, midTermStep, shortTermSize, shortTermStep, outPutFile,\
                              storeStFeatures=True, storeToCSV=True, PLOT=False)

IndexError: index -9223372036854775808 is out of bounds for axis 1 with size 6400000

In [ ]:

def dirWavFeatureExtractionNoAveraging(dirName, mtWin, mtStep, stWin, stStep):
    """
    This function extracts the mid-term features of the WAVE files of a particular folder without averaging each file.
    ARGUMENTS:
        - dirName:          the path of the WAVE directory
        - mtWin, mtStep:    mid-term window and step (in seconds)
        - stWin, stStep:    short-term window and step (in seconds)
    RETURNS:
        - X:                A feature matrix
        - Y:                A matrix of file labels
        - filenames:
    """

    allMtFeatures = numpy.array([])
    signalIndices = numpy.array([])
    processingTimes = []

    types = ('*.wav', '*.aif',  '*.aiff')
    wavFilesList = []
    for files in types:
        wavFilesList.extend(glob.glob(os.path.join(dirName, files)))

    wavFilesList = sorted(wavFilesList)

    for i, wavFile in enumerate(wavFilesList):
        [Fs, x] = audioBasicIO.readAudioFile(wavFile)            # read file
        x = audioBasicIO.stereo2mono(x)                          # convert stereo to mono
        [MidTermFeatures, _] = mtFeatureExtraction(x, Fs, round(mtWin * Fs), round(mtStep * Fs), round(Fs * stWin), round(Fs * stStep))  # mid-term feature

        MidTermFeatures = numpy.transpose(MidTermFeatures)
#        MidTermFeatures = MidTermFeatures.mean(axis=0)        # long term averaging of mid-term statistics
        if len(allMtFeatures) == 0:                # append feature vector
            allMtFeatures = MidTermFeatures
            signalIndices = numpy.zeros((MidTermFeatures.shape[0], ))
        else:
            allMtFeatures = numpy.vstack((allMtFeatures, MidTermFeatures))
            signalIndices = numpy.append(signalIndices, i * numpy.ones((MidTermFeatures.shape[0], )))

    return (allMtFeatures, signalIndices, wavFilesList)


# The following two feature extraction wrappers extract features for given audio files, however
# NO LONG-TERM AVERAGING is performed. Therefore, the output for each audio file is NOT A SINGLE FEATURE VECTOR
# but a whole feature matrix.
#
# Also, another difference between the following two wrappers and the previous is that they NO LONG-TERM AVERAGING IS PERFORMED.
# In other words, the WAV files in these functions are not used as uniform samples that need to be averaged but as sequences

def mtFeatureExtractionToFile(fileName, midTermSize, midTermStep, shortTermSize, shortTermStep, outPutFile,
                              storeStFeatures=False, storeToCSV=False, PLOT=False):
    """
    This function is used as a wrapper to:
    a) read the content of a WAV file
    b) perform mid-term feature extraction on that signal
    c) write the mid-term feature sequences to a numpy file
    """
    [Fs, x] = audioBasicIO.readAudioFile(fileName)            # read the wav file
    x = audioBasicIO.stereo2mono(x)                           # convert to MONO if required
    if storeStFeatures:
        [mtF, stF] = mtFeatureExtraction(x, Fs, round(Fs * midTermSize), round(Fs * midTermStep), round(Fs * shortTermSize), round(Fs * shortTermStep))
    else:
        [mtF, _] = mtFeatureExtraction(x, Fs, round(Fs*midTermSize), round(Fs * midTermStep), round(Fs * shortTermSize), round(Fs * shortTermStep))

    numpy.save(outPutFile, mtF)                              # save mt features to numpy file
    if PLOT:
        print "Mid-term numpy file: " + outPutFile + ".npy saved"
    if storeToCSV:
        numpy.savetxt(outPutFile+".csv", mtF.T, delimiter=",")
        if PLOT:
            print "Mid-term CSV file: " + outPutFile + ".csv saved"

    if storeStFeatures:
        numpy.save(outPutFile+"_st", stF)                    # save st features to numpy file
        if PLOT:
            print "Short-term numpy file: " + outPutFile + "_st.npy saved"
        if storeToCSV:
            numpy.savetxt(outPutFile+"_st.csv", stF.T, delimiter=",")    # store st features to CSV file
            if PLOT:
                print "Short-term CSV file: " + outPutFile + "_st.csv saved"


def mtFeatureExtractionToFileDir(dirName, midTermSize, midTermStep, shortTermSize, shortTermStep, storeStFeatures=False, storeToCSV=False, PLOT=False):
    types = (dirName + os.sep + '*.wav', )
    filesToProcess = []
    for files in types:
        filesToProcess.extend(glob.glob(files))
    for f in filesToProcess:
        outPath = f
        mtFeatureExtractionToFile(f, midTermSize, midTermStep, shortTermSize, shortTermStep, outPath, storeStFeatures, storeToCSV, PLOT)
Contact GitHub API Training Shop Blog About


In [33]:
fileName="/Volumes/Turcich-2012/AAPB_Test_Haystack_16000/cpb-aacip-500-4m91d343.16000.wav"

outPutFile="/Users/mclaugh/Desktop/af_test.csv"

[Fs, x] = audioBasicIO.readAudioFile(fileName)

mtWin=1.0
stWin=0.05

midTermSize = mtWin * Fs
midTermStep = 1.0
mtStep = 1.0


shortTermSize = stWin * Fs
shortTermStep = 0.05
stStep = 0.05


dirName="/Users/mclaugh/Desktop/test"

temp = aF.dirWavFeatureExtractionNoAveraging(dirName, mtWin, mtStep, stWin, stStep)
    

In [ ]:



import numpy as np
a = np.asarray([ [1,2,3], [4,5,6], [7,8,9] ])
numpy.savetxt("foo.csv", a, delimiter=",")




In [41]:
#for row in temp[0][:3]:
#    print(row)

[  2.60137672e-01   1.44923127e-08   3.21788057e+00   3.60266809e-01
   2.73310842e-01   2.17152323e+00   1.88225665e-03   1.53875000e-01
  -4.14698968e+01   1.00501476e+00   8.45032360e-02   1.56513386e-03
  -5.48133147e-02  -7.52069808e-02  -1.49296857e-01  -9.41188570e-02
  -4.69544363e-02  -7.63777387e-02  -1.00356631e-01  -1.16488248e-01
  -3.76955349e-02   1.63256481e-02   5.86615249e-03   1.61733086e-02
   1.27676569e-02   1.27862990e-02   9.49753729e-03   1.36856924e-02
   3.63967032e-03   9.44110939e-03   9.40778610e-03   1.87019836e-02
   5.95887310e-03   7.91065855e-03   2.85161072e-02   2.62084689e-09
   2.19010196e-01   6.70126874e-03   3.92828545e-03   7.73011742e-02
   5.22948897e-04   4.39209161e-02   3.05457989e-01   1.09166949e-01
   1.16910342e-01   1.61303368e-01   1.46114071e-01   1.75082528e-01
   1.47623915e-01   1.69070758e-01   1.13362450e-01   1.30359284e-01
   1.35828034e-01   1.33847347e-01   1.51926262e-01   1.08085052e-02
   3.28337136e-03   8.71541305e-03

In [47]:
temp[1][99]



0.0

In [40]:
len(temp[0][99])

68

In [34]:
dirName="/Users/mclaugh/Desktop/test"


aF.mtFeatureExtractionToFileDir(dirName, midTermSize, \
    midTermStep, shortTermSize, shortTermStep, \
    storeStFeatures=False, storeToCSV=False, PLOT=False)

IndexError: index -9223372036854775808 is out of bounds for axis 1 with size 6400000

## Extract 64 Mel-frequency cepstral coefficients with Librosa
- fft window size = 2048

In [ ]:
import librosa
import pandas as pd  
import numpy as np
import os

from __future__ import print_function  # for Python 3-style printing

In [2]:
## Takes WAV file path and returns a pandas dataframe containing normalized MFCC data

def wav_to_mfcc(audio_path):
    y, sr = librosa.load(audio_path)
    # Let's make a mel-scaled power (energy-squared) spectrogram
    S = librosa.feature.melspectrogram(y, sr=sr, n_mels=64, n_fft=2048)
    # Convert to log scale (dB). We'll use the peak power as reference.
    log_S = librosa.logamplitude(S, ref_power=np.max)
    log_S_rotated=np.rot90(log_S,3)
    log_S_df=pd.pandas.DataFrame(log_S_rotated)
    log_S_df['Value']=audio_path.split("/")[-1].split("|")[0]
    log_S_df['Basename']=audio_path.split("/")[-1].split("|")[-1].strip(".wav").strip(".mp4").strip(".mp3")
    return log_S_df

## Takes WAV file path and writes MFCC data to CSV in the current working directory

def mfcc_out(audio_path):
    mfcc_1=wav_to_mfcc(audio_path)
    mfcc_1.to_csv(audio_path.replace('.wav','')+".mfcc.csv", index=False)


In [3]:
## Change to directory containing speech tag excerpts in WAV form

wav_dir = os.path.expanduser('~/Dropbox/test_set_50_clips/')
os.chdir(wav_dir)
filenames=os.listdir('./')

## 

for filename in filenames:
    if ".wav" in filename:
        if filename.replace(".wav",".mfcc.csv") not in filenames:
            try:
                mfcc_out(filename)
            except:
                print("error: "+filename)


## Extract 13 Mel-frequency cepstral coefficients + 13 deltas + 13 delta-deltas
- fft window size = 2048

In [4]:
## Takes WAV file path and returns a pandas dataframe containing 13 normalized MFCCs + deltas + delta-deltas

def wav_to_mfcc_deltas(audio_path):
    y, sr = librosa.load(audio_path)
    S = librosa.feature.melspectrogram(y, sr=sr, n_mels=128, n_fft=2048)
    log_S = librosa.logamplitude(S, ref_power=np.max)
    mfcc = librosa.feature.mfcc(S=log_S, n_mfcc=13)
    delta_mfcc  = librosa.feature.delta(mfcc)
    delta2_mfcc = librosa.feature.delta(mfcc, order=2)
    mfcc_all=pd.concat([pd.DataFrame(mfcc),pd.DataFrame(delta_mfcc),pd.DataFrame(delta2_mfcc)])
    mfcc_rotated=np.rot90(mfcc_all,3)
    mfcc_df=pd.pandas.DataFrame(mfcc_rotated)
    mfcc_df['Value']=audio_path.split("/")[-1].split("|")[0]
    mfcc_df['Basename']=audio_path.split("/")[-1].split("|")[-1].strip(".wav").strip(".mp4").strip(".mp3")
    return mfcc_df

## Takes WAV file path and writes MFCC data to CSV in the current working directory

def mfcc_out_deltas(audio_path):
    mfcc_1=wav_to_mfcc(audio_path)
    mfcc_1.to_csv(audio_path.replace('.wav','')+"_deltas.mfcc.csv", index=False)


In [5]:
wav_dir = os.path.expanduser('~/Dropbox/test_set_50_clips/')
os.chdir(wav_dir)
filenames=os.listdir('./')


for filename in filenames:
    if ".wav" in filename:
        if filename.replace(".wav","_deltas.mfcc.csv") not in filenames:
            try:
                mfcc_out_deltas(filename)
            except:
                print("error : "+filename)
